# Scientific Computing Coursework

## Ode solvers

Starting with ODE solvers, I coded up a function called "solve_to" which takes an ODE and solves it using either the Euler Method or the Runga Kutta 4 method. The function created takes seven inputs with the first being the ODE. The first thing to note with this is that in every ODE and PDE that I coded up in a function, I added "t" as the first input even if there was no time variable in the actual equation. This is so that all my functions are consistent in case I end up with an ODE or PDE that does end up requiring time as an input. An input that i had to add later on down the line as i tried more complex functions was "args" which takes the arguments of the ODE that is inputted and passes them through the ODE in the function. This is useful as we want to use the ODE function in our solver and not just the ODE at a singular point. The figure below is showing a plot of both the Euler method and the Runga Kutta 4 method solving an   

In [ ]:
from IVPODES.py import solve_to
from matplotlib.pyplot import pl